In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.applications import ResNet50
import tensorflow.keras.backend as K
from sklearn.utils import class_weight

2024-11-07 19:53:19.956115: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-07 19:53:19.981793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731027200.004279  231737 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731027200.010911  231737 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-07 19:53:20.034920: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
print("TensorFlow version:", tf.__version__)
print("Available GPUs:", tf.config.list_physical_devices('GPU')) 

TensorFlow version: 2.18.0
Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:8', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:9', device_type='GPU')]


In [3]:
current_path = os.getcwd()
data_dir = current_path + '/ISIC2019Dataset/ISIC_2019_data_dir/'
training_directory_path = '/ISIC2019Dataset/ISIC_2019_Training_Input/'
training_metadata_path = '/ISIC2019Dataset/ISIC_2019_Training_Metadata.csv'
ground_truth_path = '/ISIC2019Dataset/ISIC_2019_Training_GroundTruth.csv'
test_directory_path = '/ISIC2019Dataset/ISIC_2019_Test_Input/'
test_metadata_path = '/ISIC2019Dataset/ISIC_2019_Test_Metadata.csv'

training_metadata_table = pd.read_csv(current_path + training_metadata_path)
test_metadata_table = pd.read_csv(current_path + test_metadata_path)
ground_truth_table = pd.read_csv(current_path + ground_truth_path)

LABELS

- MEL - Melanoma
- NV - Melanocytic nevus
- BCC - Basal cell carcinoma
- AK - Actinic keratosis
- BKL - Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)
- DF - Dermatofibroma
- VASC - Vascular lesion
- SCC - Squamous cell carcinoma
- UNK - None of the above

In [4]:
training_metadata_table['age_approx'].fillna((training_metadata_table['age_approx'].mean()), inplace=True)
training_metadata_table.dropna(subset=['anatom_site_general', 'lesion_id', 'sex'], inplace=True)
training_metadata_table.isnull().sum()

/tmp/ipykernel_231737/174194398.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  training_metadata_table['age_approx'].fillna((training_metadata_table['age_approx'].mean()), inplace=True)


image                  0
age_approx             0
anatom_site_general    0
lesion_id              0
sex                    0
dtype: int64

In [5]:
classes=['NV','BCC','AK','BKL','DF','VASC','SCC','MEL']

ground_truth_table['label'] = ground_truth_table[classes].idxmax(axis=1)
ground_truth_table.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK,label
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NV
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,MEL


In [6]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.array(classes),  
    y=ground_truth_table["label"]
)
class_weights_dict = dict(enumerate(class_weights))
class_weights_dict

{0: 0.24593203883495146,
 1: 0.9528663857959675,
 2: 3.652104959630911,
 3: 1.2066977896341464,
 4: 13.248430962343097,
 5: 12.515316205533598,
 6: 5.041998407643312,
 7: 0.7002156125608138}

In [7]:

# for c in classes:
#     # folders inside train_dir
#     class_train_dir = os.path.join(data_dir, c)
#     if not os.path.exists(class_train_dir):
#         os.mkdir(class_train_dir)

#     # folders inside val_dir
#     class_val_dir = os.path.join(data_dir, c)
#     if not os.path.exists(class_val_dir):
#         os.mkdir(class_val_dir)

In [8]:
# ground_truth_table.set_index('image', inplace=True)
# folder = os.listdir(current_path + training_directory_path)
# image_list = list(ground_truth_table.index)

# for image in image_list:
    
#     fname = image + '.jpg'
#     label = ground_truth_table.loc[image,'label']
    
#     if fname in folder:
#         src = os.path.join(current_path + training_directory_path, fname)
#         dst = os.path.join(data_dir, label, fname)
#         shutil.copyfile(src, dst)

## Train Test Split & Data Augmentation

In [9]:
target_size = (224,224)
image_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.15)

train_batches = image_gen.flow_from_directory(data_dir,
                                              target_size=target_size,
                                              color_mode='rgb',
                                              batch_size=32,
                                              class_mode='categorical',
                                              subset="training")

val_batches = image_gen.flow_from_directory(data_dir,
                                            target_size=target_size,
                                            color_mode='rgb',
                                            batch_size=32,
                                            class_mode='categorical',
                                            shuffle=True,
                                            subset="validation")

print(train_batches.class_indices)

Found 21535 images belonging to 8 classes.
Found 3796 images belonging to 8 classes.
{'AK': 0, 'BCC': 1, 'BKL': 2, 'DF': 3, 'MEL': 4, 'NV': 5, 'SCC': 6, 'VASC': 7}


## Transfer Learning using ResNet50 Model

In [10]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers[:-50]:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,kernel_regularizer='l2',activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(8, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.summary()

W0000 00:00:1731027210.386949  231737 gpu_device.cc:2277] Unable to enable peer access between device ordinals 0 and 9, status: INTERNAL: failed to enable peer access from 0x5599bcf75920 to 0x5599c2265900: INTERNAL: CUDA error: : CUDA_ERROR_TOO_MANY_PEERS: peer mapping resources exhausted
W0000 00:00:1731027210.428256  231737 gpu_device.cc:2277] Unable to enable peer access between device ordinals 1 and 9, status: INTERNAL: failed to enable peer access from 0x5599bd8b9070 to 0x5599c2265900: INTERNAL: CUDA error: : CUDA_ERROR_TOO_MANY_PEERS: peer mapping resources exhausted
W0000 00:00:1731027210.468891  231737 gpu_device.cc:2277] Unable to enable peer access between device ordinals 2 and 9, status: INTERNAL: failed to enable peer access from 0x5599be1ce1e0 to 0x5599c2265900: INTERNAL: CUDA error: : CUDA_ERROR_TOO_MANY_PEERS: peer mapping resources exhausted
W0000 00:00:1731027210.499871  231737 gpu_device.cc:2277] Unable to enable peer access between device ordinals 3 and 9, status: IN

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 24,114,312 (91.99 MB)

 Trainable params: 4,992,264 (19.04 MB)

 Non-trainable params: 19,122,048 (72.94 MB)

### Training Model 

In [11]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy
# set_global_policy('mixed_float16')


# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# # Set GPU memory growth to avoid allocating all GPU memory at once
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#     for gpu in gpus:
#         try:
#             tf.config.experimental.set_memory_growth(gpus[0], True)
#             tf.config.experimental.set_virtual_device_configuration(gpus[1],
#             [tf.config.LogicalDeviceConfiguration(memory_limit=1024 * 8)]  
#         )
#         except RuntimeError as e:
#             print(e)


# Custom metrics definitions
def recall_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def top_3_accuracy(y_true, y_pred):
    # y_pred = tf.cast(y_pred, tf.float32)
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

# Function to compute sample weights
def compute_sample_weights(labels, class_weights):
    sample_weights = np.array([class_weights[np.argmax(label)] for label in labels])
    return sample_weights

def generator_with_sample_weights(generator, class_weights):
    while True:
        x, y = next(generator)
        sample_weights = compute_sample_weights(y, class_weights)
        yield x, y, sample_weights

batch_size = 20
num_train_samples = len(train_batches)
num_val_samples = len(val_batches)

# Steps per epoch
train_steps = int(np.ceil(num_train_samples / batch_size))
val_steps = int(np.ceil(num_val_samples / batch_size))

train_batches_with_weights = generator_with_sample_weights(train_batches, class_weights_dict)
valid_batches_with_weights = generator_with_sample_weights(val_batches, class_weights_dict)


model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=[categorical_accuracy, recall_m, precision_m, f1_m, top_3_accuracy])

checkpoint = ModelCheckpoint('resnet50_checkpoint.keras', monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=10, verbose=1, mode='max', min_lr=0.00001)

callbacks_list = [checkpoint, reduce_lr]

history = model.fit(
    train_batches_with_weights,
    validation_data=valid_batches_with_weights,
    steps_per_epoch=int(train_steps),
    validation_steps=int(val_steps),
    epochs=30,
    callbacks=callbacks_list
)

# Save the final model
model.save('resnet50_full_model_after_training.keras')

Epoch 1/30


I0000 00:00:1731027234.054425  232114 service.cc:148] XLA service 0x7f46c00022c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731027234.054461  232114 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027234.054466  232114 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027234.054468  232114 service.cc:156]   StreamExecutor device (2): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027234.054470  232114 service.cc:156]   StreamExecutor device (3): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027234.054472  232114 service.cc:156]   StreamExecutor device (4): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027234.054474  232114 service.cc:156]   StreamExecutor device (5): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027234.054476  232114 service.cc:156]   StreamExecutor device (6): Tesla T4, Compute Capability 7.5
I0000 00:00:1731027

  2/216 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - categorical_accuracy: 0.0000e+00 - f1_m: 0.0000e+00 - loss: 30.6598 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00 - top_3_accuracy: 0.3000   

I0000 00:00:1731027245.120867  232114 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - categorical_accuracy: 0.2751 - f1_m: 0.2108 - loss: 21.9254 - precision_m: 0.3106 - recall_m: 0.1677 - top_3_accuracy: 0.5542
Epoch 1: val_categorical_accuracy improved from -inf to 0.49737, saving model to resnet50_checkpoint.keras
216/216 ━━━━━━━━━━━━━━━━━━━━ 88s 309ms/step - categorical_accuracy: 0.2757 - f1_m: 0.2115 - loss: 21.9062 - precision_m: 0.3112 - recall_m: 0.1683 - top_3_accuracy: 0.5546 - val_categorical_accuracy: 0.4974 - val_f1_m: 0.4864 - val_loss: 13.4542 - val_precision_m: 0.5011 - val_recall_m: 0.4737 - val_top_3_accuracy: 0.7474 - learning_rate: 1.0000e-05
Epoch 2/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 238ms/step - categorical_accuracy: 0.4823 - f1_m: 0.4744 - loss: 13.7534 - precision_m: 0.5205 - recall_m: 0.4392 - top_3_accuracy: 0.7510
Epoch 2: val_categorical_accuracy did not improve from 0.49737
216/216 ━━━━━━━━━━━━━━━━━━━━ 59s 272ms/step - categorical_accuracy: 0.4824 - f1_m: 0.4745 - loss: 13.7504 - precision_m: 0.52

2024-11-07 20:03:21.973198: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[5,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[5,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-07 20:03:22.286546: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[5,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[5,128,28,28]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivati

216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - categorical_accuracy: 0.5539 - f1_m: 0.5733 - loss: 11.0847 - precision_m: 0.6407 - recall_m: 0.5231 - top_3_accuracy: 0.8122

2024-11-07 20:03:31.369809: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[6,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[6,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-07 20:03:31.705293: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[6,128,28,28]{3,2,1,0}, u8[0]{0}) custom-call(f32[6,128,28,28]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivati


Epoch 10: val_categorical_accuracy did not improve from 0.54211
216/216 ━━━━━━━━━━━━━━━━━━━━ 65s 302ms/step - categorical_accuracy: 0.5540 - f1_m: 0.5734 - loss: 11.0837 - precision_m: 0.6407 - recall_m: 0.5232 - top_3_accuracy: 0.8122 - val_categorical_accuracy: 0.5239 - val_f1_m: 0.5324 - val_loss: 12.5195 - val_precision_m: 0.5611 - val_recall_m: 0.5079 - val_top_3_accuracy: 0.7713 - learning_rate: 1.0000e-05
Epoch 11/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - categorical_accuracy: 0.5864 - f1_m: 0.6015 - loss: 10.3942 - precision_m: 0.6568 - recall_m: 0.5580 - top_3_accuracy: 0.8347
Epoch 11: val_categorical_accuracy did not improve from 0.54211
216/216 ━━━━━━━━━━━━━━━━━━━━ 45s 211ms/step - categorical_accuracy: 0.5864 - f1_m: 0.6014 - loss: 10.3949 - precision_m: 0.6568 - recall_m: 0.5580 - top_3_accuracy: 0.8346 - val_categorical_accuracy: 0.5316 - val_f1_m: 0.5296 - val_loss: 11.8315 - val_precision_m: 0.5665 - val_recall_m: 0.5000 - val_top_3_accuracy: 0.8105 - learning_r

In [ ]:
# Extract metrics from history for Resnet50
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
train_recall = history.history['recall_m']
val_recall = history.history['val_recall_m']
train_precision = history.history['precision_m']
val_precision = history.history['val_precision_m']
train_f1 = history.history['f1_m']
val_f1 = history.history['val_f1_m']
train_top3_acc = history.history['top_3_accuracy']
val_top3_acc = history.history['val_top_3_accuracy']

epochs = range(1, len(train_loss) + 1)

# Plot and save the metrics
plt.figure(figsize=(20, 10))

# Plot Loss
plt.subplot(2, 3, 1)
plt.plot(epochs, train_loss, 'r-', label='Training Loss')
plt.plot(epochs, val_loss, 'b-', label='Validation Loss')
plt.title('Training and Validation Loss per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig(os.path.join(current_path, 'resnet50_training_validation_loss.png'))

# Plot Accuracy
plt.subplot(2, 3, 2)
plt.plot(epochs, train_acc, 'r-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b-', label='Validation Accuracy')
plt.title('Training and Validation Accuracy per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(os.path.join(current_path, 'resnet50_training_validation_accuracy.png'))

# Plot Recall
plt.subplot(2, 3, 3)
plt.plot(epochs, train_recall, 'r-', label='Training Recall')
plt.plot(epochs, val_recall, 'b-', label='Validation Recall')
plt.title('Training and Validation Recall per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.savefig(os.path.join(current_path, 'resnet50_training_validation_recall.png'))

# Plot Precision
plt.subplot(2, 3, 4)
plt.plot(epochs, train_precision, 'r-', label='Training Precision')
plt.plot(epochs, val_precision, 'b-', label='Validation Precision')
plt.title('Training and Validation Precision per Epoch')
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.savefig(os.path.join(current_path, 'resnet50_training_validation_precision.png'))

# Plot F1 Score
plt.subplot(2, 3, 5)
plt.plot(epochs, train_f1, 'r-', label='Training F1 Score')
plt.plot(epochs, val_f1, 'b-', label='Validation F1 Score')
plt.title('Training and Validation F1 Score per Epoch')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.legend()
plt.savefig(os.path.join(current_path, 'resnet50_training_validation_f1_score.png'))

# Plot Top 3 Accuracy
plt.subplot(2, 3, 6)
plt.plot(epochs, train_top3_acc, 'r-', label='Training Top 3 Accuracy')
plt.plot(epochs, val_top3_acc, 'b-', label='Validation Top 3 Accuracy')
plt.title('Training and validation Top 3 Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Top 3 Accuracy')
plt.legend()
plt.savefig(os.path.join(current_path, 'resnet50_training_validation_top3_acc.png'))

plt.tight_layout()
plt.savefig(os.path.join(current_path, 'resnet50all_metrics_plots.png'))
plt.show()

In [13]:
model.evaluate(val_batches)

/home/fall2019/qnd5128/miniconda3/envs/capstone_env_/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


380/380 ━━━━━━━━━━━━━━━━━━━━ 85s 211ms/step - categorical_accuracy: 0.5340 - f1_m: 0.5464 - loss: 5.7078 - precision_m: 0.5885 - recall_m: 0.5129 - top_3_accuracy: 0.8053


[5.71237850189209,
 0.5368809103965759,
 0.5167542099952698,
 0.5887309908866882,
 0.5486143827438354,
 0.8045310974121094]

In [ ]:
predictions = model.predict(val_batches, verbose=1)
predictions = predictions.argmax(axis=1)

test_labels = val_batches.classes

285/380 ━━━━━━━━━━━━━━━━━━━━ 18s 198ms/step

In [ ]:
import sklearn
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(test_labels, predictions)
print(classification_report(val_batches.classes, predictions, zero_division=0))